In [4]:
# Peaked Circuit Analysis with Pennylane and BlueQubit

import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

def read_from_qasm(path):
    with open(path, "r") as f:
        circuit = qml.from_qasm(f.read())

    return circuit

# 1. Load QASM file
qasm_path = "secret_peak.qasm"  # Replace with your path

# 2. Create device
n_qubits = 30
dev = qml.device('bluequbit.cpu', wires=n_qubits, token="kiOZkP58IdjEUN48K79qlUwnBnAfSrs0", shots = 100)

# 3. QNode with qml.from_qasm_file
@qml.qnode(dev)
def sampled_circuit():
    read_from_qasm(qasm_path)()
    return qml.sample(wires=range(n_qubits))

# 4. Run the simulation
samples = sampled_circuit()



[BQ-PYTHON-SDK][WARNING] - Beta version 0.12.0b1 of BlueQubit Python SDK is being used.


BQJobNotCompleteError: Job 8cr6Lb4f08qEw4n2 finished with status: FAILED_VALIDATION. Circuit contains more than 33 qubits, which is not supported for CPU backend with Pennylane. See https://app.bluequbit.io/docs for more details.

In [ ]:
# 5. Count bitstrings
bitstrings = ["".join(str(bit) for bit in sample) for sample in samples]
counts = Counter(bitstrings)

# 6. Get top results
most_common = counts.most_common(10)
print("Top bitstrings:")
for bitstring, count in most_common:
    print(f"{bitstring}: {count}")

# 7. Plot the histogram
def plot_top_bitstrings(counter, top_k=20):
    items = counter.most_common(top_k)
    keys = [item[0] for item in items]
    values = [item[1] for item in items]

    plt.figure(figsize=(18, 8))
    plt.bar(keys, values, color='purple')
    plt.xlabel("Bitstrings")
    plt.ylabel("Count")
    plt.title("Top Measured Bitstrings")
    plt.xticks(rotation=90)
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()

plot_top_bitstrings(counts, top_k=30)

In [3]:
import pennylane as qml
from pennylane import numpy as np
import zipfile

n_qubits = 30
shots = 256
secret_bits = np.random.randint(0, 2, size=8)
secret_string = ''.join(map(str, secret_bits))  # Şifre

dev = qml.device("default.qubit", wires=n_qubits, shots=shots)

@qml.qnode(dev)
def secret_peak_circuit():
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    for i, bit in enumerate(secret_bits):
        if bit == 1:
            qml.RZ(np.pi, wires=i)

    for i in range(0, n_qubits - 1, 2):
        qml.CNOT(wires=[i, i+1])

    return qml.sample()

# QASM almak için tape oluştur
with qml.tape.QuantumTape() as tape:
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    for i, bit in enumerate(secret_bits):
        if bit == 1:
            qml.RZ(np.pi, wires=i)
    for i in range(0, n_qubits - 1, 2):
        qml.CNOT(wires=[i, i+1])

# QASM çıktı
qasm_str = tape.to_openqasm()

# QASM dosyası yaz
with open("secret_peak.qasm", "w") as f:
    f.write(qasm_str)

# ZIP'e koy
with zipfile.ZipFile("secret_peak_qasm.zip", 'w') as zipf:
    zipf.write("secret_peak.qasm")

# (Şifreyi gizli tutmak için terminal çıktısını isteğe bağlı bırak)
print("QASM ve ZIP dosyası oluşturuldu.")
# print("Şifre:", secret_string)


QASM ve ZIP dosyası oluşturuldu.


In [5]:
print("Şifre:", secret_string)

Şifre: 11000010
